# Ejercicio 04: Evaluación de un Sistema de Recuperación de Información

El objetivo de este ejercicio es evaluar la efectividad de un sistema de recuperación de información utilizando métricas como *precisión*, *recall*, *F1-score*, *Mean Average Precision (MAP)* y *Normalized Discounted Cumulative Gain (nDCG)*.

Seguirás los siguientes pasos:

1. Proporcionar un Conjunto de Datos:


In [5]:
import xml.etree.ElementTree as XmlParser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
import pandas as pd

In [6]:
# Lectura y procesamiento de datos desde XML
xml_document = XmlParser.parse('/content/sample_data/03ranking_corpus.xml')
xml_root = xml_document.getroot()

# Crear base de datos a partir del XML
dataset = []
for node in xml_root.findall('document'):
    identifier = node.get('id')
    heading = node.find('title').text
    tags = node.find('keywords').text.split(', ')
    dataset.append({
        'document_id': identifier,
        'heading': heading,
        'tags': tags
    })

In [7]:
# Mostrar información del dataset
print("Contenido del Dataset Procesado:")
for record in dataset:
    print(f"ID: {record['document_id']}, Título: {record['heading']}, Etiquetas: {record['tags']}")

# Definir consultas
query_list = [
    "beneficios de la telemedicina en salud crónica",
    "impacto de la nutrición en el rendimiento académico",
    "estrategias para mejorar la salud mental en estudiantes universitarios"
]

print("\nLista de Consultas:")
for query_idx, query_text in enumerate(query_list, start=1):
    print(f"Consulta {query_idx}: {query_text}")

# Definir relevancia de los documentos para cada consulta
relevance_mapping = {
    "Query 1": ["1", "10"],   # Documentos relevantes para la consulta 1
    "Query 2": ["2", "7"],    # Documentos relevantes para la consulta 2
    "Query 3": ["13", "14", "23"]  # Documentos relevantes para la consulta 3
}

print("\nMapeo de Relevancia de Documentos:")
for query_name, relevant_docs in relevance_mapping.items():
    print(f"{query_name}: Documentos relevantes -> {relevant_docs}")


Contenido del Dataset Procesado:
ID: 1, Título: El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas., Etiquetas: ['telemedicina', 'salud crónica', 'tratamiento', 'tecnología médica']
ID: 2, Título: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes., Etiquetas: ['nutrición', 'rendimiento académico', 'salud mental', 'estudiantes']
ID: 3, Título: Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus., Etiquetas: ['amistad', 'bienestar estudiantil', 'campus', 'relaciones sociales']
ID: 4, Título: El rol de las bibliotecas universitarias en el fomento de la investigación académica., Etiquetas: ['bibliotecas universitarias', 'investigación', 'academia', 'recursos']
ID: 5, Título: Cómo los espacios verdes en los campus universitarios pueden mejorar la concentración y reducir el estrés., Etiquetas: ['espacios verdes', 'campus universitario', 'concentración', 'estrés']
ID: 

2. Calcular Resultados de Búsqueda:

In [11]:
# Combinar títulos y keywords para cada documento en un solo texto
document_texts = [" ".join([doc['heading']] + doc['tags']) for doc in dataset]

# Vectorizar los textos de los documentos y las consultas con TF-IDF
vectorizer = TfidfVectorizer()
corpus_tfidf = vectorizer.fit_transform(document_texts)
query_tfidf = vectorizer.transform(query_list)

# Calcular distancias y similitudes
euclidean_dist = euclidean_distances(query_tfidf, corpus_tfidf)
cosine_sim = cosine_similarity(query_tfidf, corpus_tfidf)

# Mostrar matrices
print("\nMatriz de Distancia Euclidiana:")
print(pd.DataFrame(
    euclidean_dist,
    index=[f"Consulta {i+1}" for i in range(len(query_list))],
    columns=[f"Documento {i+1}" for i in range(len(dataset))]
))

print("\nMatriz de Similitud Coseno:")
print(pd.DataFrame(
    cosine_sim,
    index=[f"Consulta {i+1}" for i in range(len(query_list))],
    columns=[f"Documento {i+1}" for i in range(len(dataset))]
))


Matriz de Distancia Euclidiana:
            Documento 1  Documento 2  Documento 3  Documento 4  Documento 5  \
Consulta 1     0.944752     1.299586     1.386398     1.372879     1.395867   
Consulta 2     1.345996     0.836806     1.374316     1.348691     1.385194   
Consulta 3     1.291102     1.117378     1.390055     1.392815     1.316638   

            Documento 6  Documento 7  Documento 8  Documento 9  Documento 10  \
Consulta 1     1.363431     1.286788     1.384989     1.372554      1.377949   
Consulta 2     1.349048     1.117503     1.372775     1.299199      1.374653   
Consulta 3     1.349783     1.062958     1.333769     1.398851      1.349887   

            ...  Documento 21  Documento 22  Documento 23  Documento 24  \
Consulta 1  ...      1.387553      1.385360      1.300964      1.394407   
Consulta 2  ...      1.265502      1.373302      1.330045      1.392619   
Consulta 3  ...      1.379821      1.312753      1.253845      1.188927   

            Documento 25  Do

In [12]:
# Ordenar y mostrar resultados por consulta
results = {}
for query_idx, query_text in enumerate(query_list):
    print(f"\nConsulta {query_idx + 1}: {query_text}")

    # Ordenar resultados por distancia euclidiana (menor distancia primero)
    euclidean_sorted = euclidean_dist[query_idx].argsort()
    euclidean_results = [dataset[idx]['document_id'] for idx in euclidean_sorted]
    print("\nDocumentos ordenados por Distancia Euclidiana:")
    for idx in euclidean_sorted:
        print(f"ID Documento: {dataset[idx]['document_id']}, Distancia: {euclidean_dist[query_idx][idx]:.4f}")

    # Ordenar resultados por similitud coseno (mayor similitud primero)
    cosine_sorted = cosine_sim[query_idx].argsort()[::-1]
    cosine_results = [dataset[idx]['document_id'] for idx in cosine_sorted]
    print("\nDocumentos ordenados por Similitud Coseno:")
    for idx in cosine_sorted:
        print(f"ID Documento: {dataset[idx]['document_id']}, Similitud: {cosine_sim[query_idx][idx]:.4f}")

    # Guardar resultados en un diccionario
    results[f"Consulta {query_idx + 1}"] = {
        "Distancia Euclidiana": euclidean_results,
        "Similitud Coseno": cosine_results
    }

# Mostrar resultados finales
print("\nResultados completos por consulta:")
for consulta, resultados in results.items():
    print(f"{consulta}:")
    print(f"  Distancia Euclidiana: {resultados['Distancia Euclidiana']}")
    print(f"  Similitud Coseno: {resultados['Similitud Coseno']}")



Consulta 1: beneficios de la telemedicina en salud crónica

Documentos ordenados por Distancia Euclidiana:
ID Documento: 1, Distancia: 0.9448
ID Documento: 26, Distancia: 1.2088
ID Documento: 7, Distancia: 1.2868
ID Documento: 2, Distancia: 1.2996
ID Documento: 23, Distancia: 1.3010
ID Documento: 13, Distancia: 1.3053
ID Documento: 11, Distancia: 1.3083
ID Documento: 14, Distancia: 1.3085
ID Documento: 15, Distancia: 1.3158
ID Documento: 12, Distancia: 1.3511
ID Documento: 20, Distancia: 1.3588
ID Documento: 6, Distancia: 1.3634
ID Documento: 27, Distancia: 1.3674
ID Documento: 29, Distancia: 1.3698
ID Documento: 16, Distancia: 1.3722
ID Documento: 9, Distancia: 1.3726
ID Documento: 4, Distancia: 1.3729
ID Documento: 17, Distancia: 1.3773
ID Documento: 10, Distancia: 1.3779
ID Documento: 8, Distancia: 1.3850
ID Documento: 28, Distancia: 1.3852
ID Documento: 22, Distancia: 1.3854
ID Documento: 25, Distancia: 1.3855
ID Documento: 3, Distancia: 1.3864
ID Documento: 19, Distancia: 1.3865


3. Calcular las Métricas de Evaluación:

In [18]:
from sklearn.metrics import ndcg_score

# Función para evaluar los resultados de recuperación
def evaluar_sistema(resultados_ranking, documentos_relevantes, top_k=5):
    # Documentos relevantes encontrados en el top-k
    encontrados_relevantes = sum(1 for doc in resultados_ranking[:top_k] if doc in documentos_relevantes)
    total_relevantes = len(documentos_relevantes)

    # Precisión en el top-k
    precision_top_k = encontrados_relevantes / top_k if top_k > 0 else 0

    # Recall (Tasa de Recuperación)
    tasa_recuperacion = encontrados_relevantes / total_relevantes if total_relevantes > 0 else 0

    # F1-score como balance entre precisión y recall
    f1_balance = (2 * precision_top_k * tasa_recuperacion) / (precision_top_k + tasa_recuperacion) if (precision_top_k + tasa_recuperacion) > 0 else 0

    # Mean Average Precision (MAP)
    acumulado_precisiones = 0
    relevantes_acumulados = 0
    for posicion, documento in enumerate(resultados_ranking[:top_k]):
        if documento in documentos_relevantes:
            relevantes_acumulados += 1
            acumulado_precisiones += relevantes_acumulados / (posicion + 1)
    promedio_precision = acumulado_precisiones / total_relevantes if total_relevantes > 0 else 0

    # nDCG: Normalized Discounted Cumulative Gain
    relevancias_binarias = [1 if doc in documentos_relevantes else 0 for doc in resultados_ranking[:top_k]]
    ndcg_valor = ndcg_score([relevancias_binarias], [list(range(top_k))], k=top_k)

    return precision_top_k, tasa_recuperacion, f1_balance, promedio_precision, ndcg_valor

# Juicios de relevancia para las consultas
relevancia_esperada = {
    "Consulta 1": ["1", "10"],
    "Consulta 2": ["2", "7"],
    "Consulta 3": ["13", "14", "23"]
}

In [19]:
# Evaluar los sistemas para cada consulta y mostrar resultados
print("\nEvaluación de Sistemas de Recuperación:")
for numero_consulta, texto_consulta in enumerate(query_list, start=1):
    documentos_relevantes = relevancia_esperada[f"Consulta {numero_consulta}"]

    print(f"\nConsulta {numero_consulta}: '{texto_consulta}'")

    # Resultados para Distancia Euclidiana
    ranking_euclidiana = results[f"Consulta {numero_consulta}"]["Distancia Euclidiana"]
    precision_eu, recall_eu, f1_eu, map_eu, ndcg_eu = evaluar_sistema(ranking_euclidiana, documentos_relevantes)
    print(f"\nDistancia Euclidiana:")
    print(f"Prec@k: {precision_eu:.4f}, Recall: {recall_eu:.4f}, F1-score: {f1_eu:.4f}, MAP: {map_eu:.4f}, nDCG: {ndcg_eu:.4f}")

    # Resultados para Similitud Coseno
    ranking_coseno = results[f"Consulta {numero_consulta}"]["Similitud Coseno"]
    precision_cos, recall_cos, f1_cos, map_cos, ndcg_cos = evaluar_sistema(ranking_coseno, documentos_relevantes)
    print(f"\nSimilitud Coseno:")
    print(f"Prec@k: {precision_cos:.4f}, Recall: {recall_cos:.4f}, F1-score: {f1_cos:.4f}, MAP: {map_cos:.4f}, nDCG: {ndcg_cos:.4f}")

    # Resumen comparativo
    mejor_sistema = "Distancia Euclidiana" if f1_eu > f1_cos else "Similitud Coseno" if f1_cos > f1_eu else "Empate"
    print(f"\nMejor desempeño: {mejor_sistema}")


Evaluación de Sistemas de Recuperación:

Consulta 1: 'beneficios de la telemedicina en salud crónica'

Distancia Euclidiana:
Prec@k: 0.2000, Recall: 0.5000, F1-score: 0.2857, MAP: 0.5000, nDCG: 0.3869

Similitud Coseno:
Prec@k: 0.2000, Recall: 0.5000, F1-score: 0.2857, MAP: 0.5000, nDCG: 0.3869

Mejor desempeño: Empate

Consulta 2: 'impacto de la nutrición en el rendimiento académico'

Distancia Euclidiana:
Prec@k: 0.4000, Recall: 1.0000, F1-score: 0.5714, MAP: 1.0000, nDCG: 0.5013

Similitud Coseno:
Prec@k: 0.4000, Recall: 1.0000, F1-score: 0.5714, MAP: 1.0000, nDCG: 0.5013

Mejor desempeño: Empate

Consulta 3: 'estrategias para mejorar la salud mental en estudiantes universitarios'

Distancia Euclidiana:
Prec@k: 0.4000, Recall: 0.6667, F1-score: 0.5000, MAP: 0.5556, nDCG: 0.5438

Similitud Coseno:
Prec@k: 0.4000, Recall: 0.6667, F1-score: 0.5000, MAP: 0.5556, nDCG: 0.5438

Mejor desempeño: Empate


4. Análisis y Comparación:

In [20]:
print("\nComparativa de Desempeño entre Sistemas de Recuperación:")

for indice, consulta_texto in enumerate(query_list):
    documentos_relevantes = juicios_relevancia[f"Consulta {indice + 1}"]

    print(f"\nEvaluación de la Consulta {indice + 1}: '{consulta_texto}'")

    # Recuperar los resultados para ambos métodos
    resultados_euclidiana = results[f"Consulta {indice + 1}"]["Distancia Euclidiana"]
    resultados_coseno = results[f"Consulta {indice + 1}"]["Similitud Coseno"]

    # Calcular métricas para Distancia Euclidiana
    metrica_euclidiana = calcular_metricas(resultados_euclidiana, documentos_relevantes)
    precision_euc, recall_euc, f1_euc, map_euc, ndcg_euc = metrica_euclidiana

    # Calcular métricas para Similitud Coseno
    metrica_coseno = calcular_metricas(resultados_coseno, documentos_relevantes)
    precision_cos, recall_cos, f1_cos, map_cos, ndcg_cos = metrica_coseno

    # Análisis basado en F1-score
    print("\nResultados Comparativos:")
    if f1_euc > f1_cos:
        print("→ El sistema basado en Distancia Euclidiana ofrece un mejor desempeño para esta consulta.")
    elif f1_cos > f1_euc:
        print("→ El sistema basado en Similitud Coseno ofrece un mejor desempeño para esta consulta.")
    else:
        print("→ Ambos sistemas presentan un desempeño equivalente en esta consulta.")

    # Mostrar métricas detalladas para ambos sistemas
    print("\nMétricas Detalladas:")
    print(f"Distancia Euclidiana - Prec@k: {precision_euc:.4f}, Recall: {recall_euc:.4f}, F1-score: {f1_euc:.4f}, MAP: {map_euc:.4f}, nDCG: {ndcg_euc:.4f}")
    print(f"Similitud Coseno    - Prec@k: {precision_cos:.4f}, Recall: {recall_cos:.4f}, F1-score: {f1_cos:.4f}, MAP: {map_cos:.4f}, nDCG: {ndcg_cos:.4f}")

    # Adicional: Resumen en una estructura legible
    print(f"\nResumen para la Consulta {indice + 1}:")
    print(f"- Mejor sistema: {'Distancia Euclidiana' if f1_euc > f1_cos else 'Similitud Coseno' if f1_cos > f1_euc else 'Empate'}")
    print(f"- F1-score: Euclidiana={f1_euc:.4f}, Coseno={f1_cos:.4f}")



Comparativa de Desempeño entre Sistemas de Recuperación:

Evaluación de la Consulta 1: 'beneficios de la telemedicina en salud crónica'

Resultados Comparativos:
→ Ambos sistemas presentan un desempeño equivalente en esta consulta.

Métricas Detalladas:
Distancia Euclidiana - Prec@k: 0.2000, Recall: 0.5000, F1-score: 0.2857, MAP: 0.5000, nDCG: 0.3869
Similitud Coseno    - Prec@k: 0.2000, Recall: 0.5000, F1-score: 0.2857, MAP: 0.5000, nDCG: 0.3869

Resumen para la Consulta 1:
- Mejor sistema: Empate
- F1-score: Euclidiana=0.2857, Coseno=0.2857

Evaluación de la Consulta 2: 'impacto de la nutrición en el rendimiento académico'

Resultados Comparativos:
→ Ambos sistemas presentan un desempeño equivalente en esta consulta.

Métricas Detalladas:
Distancia Euclidiana - Prec@k: 0.4000, Recall: 1.0000, F1-score: 0.5714, MAP: 1.0000, nDCG: 0.5013
Similitud Coseno    - Prec@k: 0.4000, Recall: 1.0000, F1-score: 0.5714, MAP: 1.0000, nDCG: 0.5013

Resumen para la Consulta 2:
- Mejor sistema: Empat